# Detecting Gaze Events

## What you will learn in this tutorial:

* how to detect saccades using the microsaccades algorithm
* how to detect fixations using the I-DT and I-VT algorithms

## Preparations

We import `pymovements` as the alias `pm` for convenience.

In [1]:
import polars as pl

import pymovements as pm

/home/krakowczyk/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's start by downloading our `ToyDataset` and loading in its data:

In [2]:
dataset = pm.Dataset('ToyDataset', path='data/ToyDataset')
dataset.download()
dataset.load()

Using already downloaded and verified file: data/ToyDataset/downloads/pymovements-toy-dataset.zip
Extracting pymovements-toy-dataset.zip to data/ToyDataset/raw


100%|██████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 397.79it/s]


Now let's do some basic preprocessing:

In [3]:
dataset.pix2deg()
dataset.pos2vel('smooth')

dataset.gaze[0].frame.head()

100%|██████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 560.91it/s]


text_id,page_id,time,x_right_pix,y_right_pix,x_right_pos,y_right_pos,y_right_vel,x_right_vel
i64,i64,f64,f64,f64,f64,f64,f64,f64
0,1,1.988145e6,206.8,152.4,-10.697598,-8.852399,-3.639106,1.207626
0,1,1.988146e6,206.9,152.1,-10.695183,-8.859678,-7.278067,2.415272
0,1,1.988147e6,207.0,151.8,-10.692768,-8.866956,-5.256267,1.610194
0,1,1.988148e6,207.1,151.7,-10.690352,-8.869381,-4.447465,0.402548
0,1,1.988149e6,207.0,151.5,-10.692768,-8.874233,-3.234462,0.402561


## Detecting Events

*pymovements* provides a range of event detection methods for several types of gaze events.

See the reference for [pymovements.events](https://pymovements.readthedocs.io/en/latest/reference/pymovements.events.html) to get an overview of all the supported methods.

For this tutorial we will use the `microsaccades` algorithm for detecting saccades and the I-DT and I-VT algorithms for detecting fixations.

We start out by detecting saccades.

In [4]:
dataset.detect_events('microsaccades', minimum_duration=12)

20it [00:00, 116.62it/s]


The detected events are added as rows with the name `saccade` to the event dataframe:

In [5]:
dataset.events[0].frame.head()

text_id,page_id,name,onset,offset,duration
i64,i64,str,i64,i64,i64
0,1,"""saccade""",1988323,1988337,14
0,1,"""saccade""",1988547,1988567,20
0,1,"""saccade""",1988737,1988760,23
0,1,"""saccade""",1988765,1988777,12
0,1,"""saccade""",1989014,1989031,17


Next we will detect fixations using the I-DT and I-VT algorithms.

To be able to differentiate between the detected events, we specify custom event names for each algorithm:

dataset.detect_events('idt', dispersion_threshold=2.7)
dataset.detect_events('ivt', velocity_threshold=20)

In [8]:
dataset.events[0].frame.filter(pl.col('name') == 'fixation').head()

text_id,page_id,name,onset,offset,duration
i64,i64,str,i64,i64,i64
0,1,"""fixation""",1988145,1988563,418
0,1,"""fixation""",1988564,1988750,186
0,1,"""fixation""",1988751,1989178,427
0,1,"""fixation""",1989179,1989436,257
0,1,"""fixation""",1989437,1989600,163


This has added new rows with the fixation events to the event dataframe.

## What you have learned in this tutorial:

* detecting saccades by using the microsaccades algorithm
* detecting fixations by using the I-DT and I-VT algorithms